In [2]:
import pandas as pd
import requests

from api_keys import geoapify_key

In [2]:

# set up a parameters dictionary
params = {
    "format":"json",
    "apiKey": geoapify_key   
}

# Set base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [3]:
# Load the data
air_pollution_data = "../Resources/global_air_pollution.csv"

# Specify the encoding parameter when reading CSV files
air_pollution_df = pd.DataFrame(pd.read_csv(air_pollution_data, encoding='latin-1'))

air_pollution_df

,ï»¿Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category
0,Russian Federation,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate
1,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good
2,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate
3,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good
4,France,Punaauia,22,Good,0,Good,22,Good,0,Good,6,Good
...,...,...,...,...,...,...,...,...,...,...,...,...
23458,India,Gursahaiganj,184,Unhealthy,3,Good,154,Unhealthy,2,Good,184,Unhealthy
23459,France,Sceaux,50,Good,1,Good,20,Good,5,Good,50,Good
23460,India,Mormugao,50,Good,1,Good,22,Good,1,Good,50,Good
23461,United States of America,Westerville,71,Moderate,1,Good,44,Good,2,Good,71,Moderate


In [4]:
air_pollution_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23463 entries, 0 to 23462
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ï»¿Country          23036 non-null  object
 1   City                23462 non-null  object
 2   AQI Value           23463 non-null  int64 
 3   AQI Category        23463 non-null  object
 4   CO AQI Value        23463 non-null  int64 
 5   CO AQI Category     23463 non-null  object
 6   Ozone AQI Value     23463 non-null  int64 
 7   Ozone AQI Category  23463 non-null  object
 8   NO2 AQI Value       23463 non-null  int64 
 9   NO2 AQI Category    23463 non-null  object
 10  PM2.5 AQI Value     23463 non-null  int64 
 11  PM2.5 AQI Category  23463 non-null  object
dtypes: int64(5), object(7)
memory usage: 2.1+ MB


In [5]:
# Convert column names to snake_case
air_pollution_df.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

# Display the DataFrame info with updated column names
print(air_pollution_df.columns)

Index(['ï»¿country', 'city', 'aqi_value', 'aqi_category', 'co_aqi_value',
       'co_aqi_category', 'ozone_aqi_value', 'ozone_aqi_category',
       'no2_aqi_value', 'no2_aqi_category', 'pm2.5_aqi_value',
       'pm2.5_aqi_category'],
      dtype='object')


In [6]:
# Remove special characters from column name
air_pollution_df.rename(columns=lambda x: x.replace('ï»¿', ''), inplace=True)

# Display the DataFrame info with updated column names
print(air_pollution_df.columns)


Index(['country', 'city', 'aqi_value', 'aqi_category', 'co_aqi_value',
       'co_aqi_category', 'ozone_aqi_value', 'ozone_aqi_category',
       'no2_aqi_value', 'no2_aqi_category', 'pm2.5_aqi_value',
       'pm2.5_aqi_category'],
      dtype='object')


In [7]:
# As we see from info() we have missing values  in Country column 
# Filling the gaps

missing_country_cities = air_pollution_df.loc[air_pollution_df.country.isna()]['city'].unique()
print({len(missing_country_cities)})
for city in missing_country_cities[:15]:
    print(f'- {city}')


{427}
- Granville
- Kingston Upon Hull
- New Waterford
- Kingstown
- Nanakuli
- Lavagna
- Ladispoli
- Dong Hoi
- Nettuno
- Puebloviejo
- Fiumicino
- Nishinomiya
- Carteret
- Ercolano
- Bolama


In [8]:
# Fill missing values in the 'country' column with 'Unknown'
air_pollution_df['country'].fillna('Unknown', inplace=True)

# Verify if there are any remaining missing values in the 'country' column
missing_country_cities = air_pollution_df.loc[air_pollution_df.country.isna()]['city'].unique()
print({len(missing_country_cities)})
for city in missing_country_cities[:15]:
    print(f'- {city}')


{0}


In [9]:
# Let's drop  entries we will not be able to use with Unknown.
air_pollution_df = air_pollution_df.loc[air_pollution_df.country != 'Unknown']

In [10]:

(
    air_pollution_df
    .groupby('country')
    # Count the rows: any column without missing values
    ['aqi_category'].count()
    # Show the statistics.
    .describe()
)

count     175.000000
mean      131.634286
std       355.829019
min         1.000000
25%        10.500000
50%        28.000000
75%        80.500000
max      2872.000000
Name: aqi_category, dtype: float64

In [12]:
# Get the city's name & add row "country" to the string so geoapify finds the correct city
for index, row in air_pollution_df.iterrows():
    city = f'{row["city"]}, {row["country"]}'
    params["text"] = f"{city}"
    response = requests.get(base_url, params=params)
    response = response.json()
    
    # Check if there are any results returned by the API
    if "results" in response and response["results"]:
        # Extract latitude and longitude from the first result
        air_pollution_df.loc[index, "Lat"] = response["results"][0]["lat"]
        air_pollution_df.loc[index, "Lon"] = response["results"][0]["lon"]
        print(f"Coordinates for {city} fetched...")
    else:
        print(f"No coordinates found for {city}")

# Display sample data to confirm that the coordinates appear

/var/folders/59/r7qg_d3576j6l4465z6wt9000000gn/T/ipykernel_58208/2315058433.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  air_pollution_df.loc[index, "Lat"] = response["results"][0]["lat"]
/var/folders/59/r7qg_d3576j6l4465z6wt9000000gn/T/ipykernel_58208/2315058433.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  air_pollution_df.loc[index, "Lon"] = response["results"][0]["lon"]


Coordinates for Praskoveya, Russian Federation fetched...
Coordinates for Presidente Dutra, Brazil fetched...
Coordinates for Priolo Gargallo, Italy fetched...
Coordinates for Przasnysz, Poland fetched...
Coordinates for Punaauia, France fetched...
Coordinates for Punta Gorda, United States of America fetched...
Coordinates for Puttlingen, Germany fetched...
Coordinates for Puurs, Belgium fetched...
Coordinates for Pyatigorsk, Russian Federation fetched...
Coordinates for Qalyub, Egypt fetched...
Coordinates for Qinzhou, China fetched...
Coordinates for Raalte, Netherlands fetched...
Coordinates for Radaur, India fetched...
Coordinates for Radhan, Pakistan fetched...
Coordinates for Radovis, Republic of North Macedonia fetched...
Coordinates for Raismes, France fetched...
Coordinates for Rajgir, India fetched...
Coordinates for Ramacca, Italy fetched...
Coordinates for Phoenix, United States of America fetched...
Coordinates for Phulabani, India fetched...
Coordinates for Piaseczno, Po

Coordinates for Friendly, United States of America fetched...
Coordinates for Fuldatal, Germany fetched...
Coordinates for Furano, Japan fetched...
Coordinates for Fuzhou, China fetched...
Coordinates for Galashiels, United Kingdom of Great Britain and Northern Ireland fetched...
Coordinates for Gangelt, Germany fetched...
Coordinates for Garbsen, Germany fetched...
Coordinates for Garoua, Cameroon fetched...
Coordinates for Garowe, Somalia fetched...
Coordinates for Geertruidenberg, Netherlands fetched...
Coordinates for Geraldton, Australia fetched...
Coordinates for Germersheim, Germany fetched...
Coordinates for Girardot, Colombia fetched...
Coordinates for Glubokiy, Russian Federation fetched...
Coordinates for Golborne, United Kingdom of Great Britain and Northern Ireland fetched...
Coordinates for Gondomar, Portugal fetched...
Coordinates for Goryachevodskiy, Russian Federation fetched...
Coordinates for Govindgarh, India fetched...
Coordinates for Grand Lahou, CÃ´te d'Ivoire fe

Coordinates for Culleredo, Spain fetched...
Coordinates for Cuncolim, India fetched...
Coordinates for Curvelo, Brazil fetched...
Coordinates for Cuxhaven, Germany fetched...
Coordinates for Cypress Lake, United States of America fetched...
Coordinates for Czestochowa, Poland fetched...
Coordinates for Dalaba, Guinea fetched...
Coordinates for Dalbandin, Pakistan fetched...
Coordinates for Beaune, France fetched...
Coordinates for Bedford, United States of America fetched...
Coordinates for Begoro, Ghana fetched...
Coordinates for Beipiao, China fetched...
Coordinates for Bela Cruz, Brazil fetched...
Coordinates for Bermejo, Bolivia (Plurinational State of) fetched...
Coordinates for Bernburg, Germany fetched...
Coordinates for Berwick, United States of America fetched...
Coordinates for Betma, India fetched...
Coordinates for Beverly, United States of America fetched...
Coordinates for Bezhetsk, Russian Federation fetched...
Coordinates for Bhairab Bazar, Bangladesh fetched...
Coordin

Coordinates for Delta Del Tigre, Uruguay fetched...
Coordinates for Delvada, India fetched...
Coordinates for Desnogorsk, Russian Federation fetched...
Coordinates for Develi, Turkey fetched...
Coordinates for Dewa, India fetched...
Coordinates for Dinar, Turkey fetched...
Coordinates for Dniprorudne, Ukraine fetched...
Coordinates for Dogbo, Benin fetched...
Coordinates for Donskoye, Russian Federation fetched...
Coordinates for Dumas, United States of America fetched...
Coordinates for Dunda, United Republic of Tanzania fetched...
Coordinates for Dyersburg, United States of America fetched...
Coordinates for Eatontown, United States of America fetched...
Coordinates for Ebano, Mexico fetched...
Coordinates for El Campo, United States of America fetched...
Coordinates for El Pueblito, Mexico fetched...
Coordinates for El Torno, Bolivia (Plurinational State of) fetched...
Coordinates for Ellicott City, United States of America fetched...
Coordinates for Eminabad, Pakistan fetched...
Co

...

Coordinates for Werdohl, Germany fetched...
Coordinates for Witten, Germany fetched...
Coordinates for Xingyi, China fetched...
Coordinates for Xuanzhou, China fetched...
Coordinates for Vocklabruck, Austria fetched...
Coordinates for Waldorf, United States of America fetched...
Coordinates for Omaruru, Namibia fetched...
Coordinates for Palanga, Lithuania fetched...
Coordinates for Paralimni, Cyprus fetched...
Coordinates for Lehrte, Germany fetched...
Coordinates for Licheng, China fetched...
Coordinates for Snellville, United States of America fetched...
Coordinates for Sojat, India fetched...
Coordinates for Agoo, Philippines fetched...
Coordinates for Alpen, Germany fetched...
Coordinates for Arequipa, Peru fetched...
Coordinates for Worgl, Austria fetched...
Coordinates for Yingzhong, China fetched...
Coordinates for Yunjinghong, China fetched...
Coordinates for Zarzal, Colombia fetched...
Coordinates for Trossingen, Germany fetched...
Coordinates for Vesele, Ukraine fetched...
C

Coordinates for Parede, Portugal fetched...
Coordinates for Pittsburgh, United States of America fetched...
Coordinates for Belcesti, Romania fetched...
Coordinates for Boca Del Rio, Mexico fetched...
Coordinates for Campo De Criptana, Spain fetched...
Coordinates for Vitre, France fetched...
Coordinates for Volokonovka, Russian Federation fetched...
Coordinates for Vyborg, Russian Federation fetched...
Coordinates for Worms, Germany fetched...
Coordinates for Xanten, Germany fetched...
Coordinates for Francisco Beltrao, Brazil fetched...
Coordinates for Illintsi, Ukraine fetched...
Coordinates for Dubinino, Russian Federation fetched...
Coordinates for Enem, Russian Federation fetched...
Coordinates for Biankouma, CÃ´te d'Ivoire fetched...
Coordinates for Bobrov, Russian Federation fetched...
Coordinates for Bresso, Italy fetched...
Coordinates for Chivacoa, Venezuela (Bolivarian Republic of) fetched...
Coordinates for Piekary Slaskie, Poland fetched...
Coordinates for Srikakulam, Ind

Coordinates for Baturite, Brazil fetched...
Coordinates for Bikin, Russian Federation fetched...
Coordinates for Scottsburgh, South Africa fetched...
Coordinates for Norvenich, Germany fetched...
Coordinates for Chicago Heights, United States of America fetched...
Coordinates for Gori, Georgia fetched...
Coordinates for Guacara, Venezuela (Bolivarian Republic of) fetched...
Coordinates for Guanare, Venezuela (Bolivarian Republic of) fetched...
Coordinates for Heiloo, Netherlands fetched...
Coordinates for Epinay Sur Seine, France fetched...
Coordinates for Kropotkin, Russian Federation fetched...
Coordinates for Kualakapuas, Indonesia fetched...
Coordinates for Lakeland Highlands, United States of America fetched...
Coordinates for Leixlip, Ireland fetched...
Coordinates for American Fork, United States of America fetched...
Coordinates for Anshan, China fetched...
Coordinates for Attayyampatti, India fetched...
Coordinates for Cormeilles En Parisis, France fetched...
Coordinates for M

Coordinates for Phek, India fetched...
Coordinates for Povoa De Santo Adriao, Portugal fetched...
Coordinates for Prokhorovka, Russian Federation fetched...
Coordinates for Paramaribo, Suriname fetched...
Coordinates for Portadown, United Kingdom of Great Britain and Northern Ireland fetched...
Coordinates for Portoviejo, Ecuador fetched...
Coordinates for Rajauri, India fetched...
Coordinates for Shiyan, China fetched...
Coordinates for Strzelce Opolskie, Poland fetched...
Coordinates for San Bartolo Del Llano, Mexico fetched...
Coordinates for Rosa, Italy fetched...
Coordinates for Rutana, Burundi fetched...
Coordinates for Godhra, India fetched...
Coordinates for Tlalancaleca, Mexico fetched...
Coordinates for Tubize, Belgium fetched...
Coordinates for Urbino, Italy fetched...
Coordinates for Veternik, Serbia fetched...
Coordinates for Taggia, Italy fetched...
Coordinates for South Houston, United States of America fetched...
Coordinates for Sulzbach Rosenberg, Germany fetched...
Co

Coordinates for Kuppam, India fetched...
Coordinates for Lleida, Spain fetched...
Coordinates for Girard, United States of America fetched...
Coordinates for Harelbeke, Belgium fetched...
Coordinates for Novokubansk, Russian Federation fetched...
Coordinates for Prambanan, Indonesia fetched...
Coordinates for Rottenburg, Germany fetched...
Coordinates for Torgau, Germany fetched...
Coordinates for Trelleborg, Sweden fetched...
Coordinates for Wloszczowa, Poland fetched...
Coordinates for Ludwigshafen, Germany fetched...
Coordinates for Macaiba, Brazil fetched...
Coordinates for Creil, France fetched...
Coordinates for Maracai, Brazil fetched...
Coordinates for Tanga, United Republic of Tanzania fetched...
Coordinates for Vigo, Spain fetched...
Coordinates for Viseu, Portugal fetched...
Coordinates for Mian Channun, Pakistan fetched...
Coordinates for Wesselsbron, South Africa fetched...
Coordinates for Willow Grove, United States of America fetched...
Coordinates for Wrentham, United S

In [13]:
# Display sample data to confirm that the coordinates appear
print(response)

{'results': [{'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright'}, 'name': 'Marang', 'country': 'Malaysia', 'country_code': 'my', 'state': 'Terengganu', 'city': 'Marang', 'postcode': '21600', 'district': 'Marang', 'lon': 103.2042795, 'lat': 5.2065457, 'result_type': 'district', 'formatted': 'Marang, Terengganu, Malaysia', 'address_line1': 'Marang', 'address_line2': 'Marang, Terengganu, Malaysia', 'category': 'populated_place', 'timezone': {'name': 'Asia/Kuala_Lumpur', 'name_alt': 'Asia/Singapore', 'offset_STD': '+08:00', 'offset_STD_seconds': 28800, 'offset_DST': '+08:00', 'offset_DST_seconds': 28800}, 'plus_code': '6PQ56643+JP', 'plus_code_short': '43+JP Marang, Terengganu, Malaysia', 'rank': {'importance': 0.4813117306116221, 'popularity': 0.6828888694415939, 'confidence': 1, 'confidence_city_level': 1, 'match_type': 'full_match'}, 'place_id': '5191ef52ea12cd

In [14]:
# Display sample data to confirm that the coordinates appear
air_pollution_df.head()

,country,city,aqi_value,aqi_category,co_aqi_value,co_aqi_category,ozone_aqi_value,ozone_aqi_category,no2_aqi_value,no2_aqi_category,pm2.5_aqi_value,pm2.5_aqi_category,Lat,Lon
0,Russian Federation,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate,44.745212,44.193819
1,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-11.295918,-41.987157
2,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate,37.157930,15.186029
3,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good,53.019022,20.880403
4,France,Punaauia,22,Good,0,Good,22,Good,0,Good,6,Good,-17.592920,-149.606367


In [15]:
print(air_pollution_df.columns)

Index(['country', 'city', 'aqi_value', 'aqi_category', 'co_aqi_value',
       'co_aqi_category', 'ozone_aqi_value', 'ozone_aqi_category',
       'no2_aqi_value', 'no2_aqi_category', 'pm2.5_aqi_value',
       'pm2.5_aqi_category', 'Lat', 'Lon'],
      dtype='object')


In [26]:
# Assuming air_pollution_df is your DataFrame
num_rows, num_columns = air_pollution_df.shape

print(f"The DataFrame has {num_rows} rows and {num_columns} columns.")


The DataFrame has 23036 rows and 14 columns.


In [27]:
air_pollution_df.to_csv('../Resources/aqi_data.csv', index=False)